# **Limpieza y transformación de los datos de la tabla Store**

# Tabla de Contenido
- [TABLA STORE](#tabla-store)
  - [Columna City](#columna-city)
  - [Columna Open_Date](#Columna-open_date)
  - [Columna Zip_Code](#Columna-zip_code)
  - [Columna Phone](#Columna-phone)
  - [Columna Email](#Columna-email)
  - [Resultados y conclusiones](#Resultados-y-conclusiones)


<span style="font-size: 18px;">
En el análisis de los datos de la tabla Store, se identificaron las siguientes observaciones:

1. La tabla <b>Store</b> contiene un total de 45 registros distribuidos en 6 columnas.
1. La columna <b>City</b>  incluye 32 nombres de ciudades que requieren conversión a un formato estándar.
1. La columna <b>Open_Date</b> presenta 3 registros con fechas en un formato inválido.
1. La columna <b>Zip_Code</b> tiene 9 registros con caracteres no numéricos.
1. La columna <b>Phone</b> contiene 35 registros con caracteres no numéricos.
1. La columna <b>Email</b> cuenta con 13 correos electrónicos que no cumplen con el formato estándar (nombre@dominio.es).

A continuación, se procede  a realizar las correcciones necesarias en los datos de esta tabla para garantizar su calidad y consistencia. Esto involucra lo siguiente:

1. Consulta de información sobre las columnas, incluyendo su nombre y tipo de dato.
1. Limpieza y estandarización de los datos en la columna "City".
1. Corrección de fechas con formato inválido.
1. Eliminación de caracteres no numéricos en la columna "Zip_Code".
1. Eliminación de caracteres no numéricos en la columna "Phone".
1. Corrección de correos electrónicos con formato inválido.

</span>

In [12]:
-- # 1.
USE TFM;
SELECT *FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Store'
--BACKUP DATABASE TFM
--TO DISK = 'D:\6. NEXER\master\Solucion TFM\0. Data SQL\1. Bakup Data'

(6 rows affected)

Total execution time: 00:00:00.088

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,NUMERIC_PRECISION,NUMERIC_PRECISION_RADIX,NUMERIC_SCALE,DATETIME_PRECISION,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME
TFM,dbo,Store,Store_ID,1,NULL,NO,tinyint,NULL,NULL,3,10,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
TFM,dbo,Store,City,2,NULL,NO,nvarchar,50,100,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,Modern_Spanish_100_CI_AI,NULL,NULL,NULL
TFM,dbo,Store,Open_Date,3,NULL,NO,nvarchar,50,100,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,Modern_Spanish_100_CI_AI,NULL,NULL,NULL
TFM,dbo,Store,Zip_Code,4,NULL,NO,nvarchar,50,100,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,Modern_Spanish_100_CI_AI,NULL,NULL,NULL
TFM,dbo,Store,Phone,5,NULL,NO,nvarchar,50,100,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,Modern_Spanish_100_CI_AI,NULL,NULL,NULL
TFM,dbo,Store,Email,6,NULL,NO,nvarchar,50,100,NULL,NULL,NULL,NULL,NULL,NULL,UNICODE,NULL,NULL,Modern_Spanish_100_CI_AI,NULL,NULL,NULL


## **Columna City**
<span style="font-size: 18px;">
Para limpiar esta columna realizamos lo siguiente:

1. Creamos la función CleanToTexT: Esta función realiza lo siguiente:
    - <b>Recepción de datos:</b> Recibe como entrada una columna de interés.
    -  <b>Identificación de caracteres no deseados:</b> Utiliza la instrucción SET @Letter = PATINDEX('%[^A-Za-z _]%', @Data) para localizar la posición del primer carácter que no sea una letra, un espacio ni un guion bajo.
    - <b>Eliminación de caracteres no deseados:</b> Se inicia un bucle WHILE que elimina, con la instrucción SET @Data = STUFF(@Data, @Letter, 1, ''), el carácter no permitido ubicado en la posición @Letter. Este proceso se repite mientras @Letter > 0.
    - <b>Reemplazo de guiones bajos:</b> Una vez eliminados los caracteres no deseados, se reemplazan los guiones bajos ("___") por un espacio, utilizando SET @Data = REPLACE(@Data, '_', ' ').
    - <b>Eliminación de espacios consecutivos:</b> Finalmente, se ejecuta un segundo bucle WHILE que elimina los espacios en blanco consecutivos presentes en la cadena almacenada en @Data, asegurando que quede limpia y bien formateada.

1. Capitalizamos los nombres de las ciudades usando la función InitCap.
</span>

In [13]:
-- # 1.
CREATE FUNCTION [dbo].[CleanToText](@Data VARCHAR(100))
RETURNS VARCHAR(100)
AS 
BEGIN
    DECLARE @Letter INT;
    SET @Letter = PATINDEX('%[^A-Za-z _]%', @Data);
    WHILE @Letter > 0
    BEGIN
        SET @Data = STUFF(@Data, @Letter, 1, '');
        SET @Letter = PATINDEX('%[^A-Za-z ]%', @Data);
    END
    SET @Data = REPLACE(@Data, '_', ' ');
    WHILE CHARINDEX('  ', @Data) > 0
    BEGIN
        SET @Data = REPLACE(@Data, '  ', ' ');
    END
    
    RETURN LTRIM(RTRIM(@Data));
END
GO

Commands completed successfully.

Total execution time: 00:00:00.004

In [14]:
-- # 1.
UPDATE Store
SET City = dbo.CleanToText(City);

(45 rows affected)

Total execution time: 00:00:00.010

In [15]:
 CREATE FUNCTION [dbo].[InitCap] (@inStr VARCHAR(100))
  RETURNS VARCHAR(100)
  AS
  BEGIN
    DECLARE @outStr VARCHAR(100) = LOWER(@inStr),
         @char CHAR(1), 
         @alphanum BIT = 0,
         @len INT = LEN(@inStr),
                 @pos INT = 1;        

    -- Iterar entre todos los caracteres en la cadena de entrada
    WHILE @pos <= @len BEGIN

      -- Obtener el siguiente caracter
      SET @char = SUBSTRING(@inStr, @pos, 1);

      -- Si la posición del caracter es la 1ª, o el caracter previo no es alfanumérico
      -- convierte el caracter actual a mayúscula
      IF @pos = 1 OR @alphanum = 0
        SET @outStr = STUFF(@outStr, @pos, 1, UPPER(@char));

      SET @pos = @pos + 1;

      -- Define si el caracter actual es  non-alfanumérico
      IF ASCII(@char) <= 47 OR (ASCII(@char) BETWEEN 58 AND 64) OR
      (ASCII(@char) BETWEEN 91 AND 96) OR (ASCII(@char) BETWEEN 123 AND 126)
      SET @alphanum = 0;
      ELSE
      SET @alphanum = 1;

    END

   RETURN @outStr;         
  END
  GO

Commands completed successfully.

Total execution time: 00:00:00.002

In [16]:
UPDATE Store
SET City = dbo.InitCap(City);

(45 rows affected)

Total execution time: 00:00:00.007

## **Columna Open_Date**
<span style="font-size: 18px;">
En esta columna, las fechas con formato inválido se presentan en el formato YYYYMMDDHHmmss. Para corregir estas fechas, se utilizó la función ISDATE, que devuelve un valor de 0 cuando la fecha no está en un formato válido. Aclarado esto, se realizaron los siguientes pasos:

- Si ISDATE(Open_Date) = 0, entonces, utilizando la función CONCAT, transformamos el formato de la fecha de la siguiente manera:
    -   Extraemos el año, correspondiente a los primeros 4 caracteres, y añadimos un guion (-).
    -   Extraemos el mes, correspondiente al quinto y sexto carácter, y añadimos otro guion (-).
    -   Extraemos el día, correspondiente al séptimo y octavo carácter, y añadimos un espacio ( ).
    -   Extraemos la hora, correspondiente al noveno y décimo carácter, y añadimos dos puntos (:).
    -   Extraemos los minutos, correspondientes al undécimo y duodécimo carácter, y añadimos otros dos puntos (:).
    - Finalmente, agregamos los segundos, correspondientes a los últimos 2 caracteres.

Con estos pasos, las fechas en formato incorrecto se convierten al formato estándar YYYY-MM-DD HH:mm:ss.
</span>

In [17]:
-- # 3.
UPDATE Store
SET Open_Date = CONCAT(
        LEFT(Open_Date, 4), '-',            -- Año
        SUBSTRING(Open_Date, 5, 2), '-',    -- Mes
        SUBSTRING(Open_Date, 7, 2), ' ',    -- Día
        SUBSTRING(Open_Date, 9, 2), ':',    -- Hora
        SUBSTRING(Open_Date, 11, 2), ':',   -- Minuto
        RIGHT(Open_Date, 2)                 -- Segundo
    )
WHERE ISDATE(Open_Date) = 0;

(3 rows affected)

Total execution time: 00:00:00.003

## **Columna Zip_Code**

<span style="font-size: 18px;">
En esta columna nos enfocamos en eliminar los caracteres no numéricos. Para ello, creamos la función CleanToNum, que utiliza PATINDEX para identificar los caracteres no numéricos y los elimina mediante un ciclo WHILE, empleando la función STUFF. Una vez que no quedan caracteres no numéricos en la cadena, se eliminan los espacios al inicio y al final para garantizar un resultado limpio.
</span>

In [18]:
-- # 4.
CREATE FUNCTION [dbo].[CleanToNum](@Data VARCHAR(100))
RETURNS VARCHAR(100)
AS 
BEGIN
    DECLARE @posicion INT;
    SET @posicion = PATINDEX('%[^0-9]%', @Data);
    WHILE @posicion > 0
    BEGIN
        SET @Data = STUFF(@Data, @posicion,1,'');
        SET @posicion = PATINDEX('%[^0-9]%', @Data);
    END
    RETURN LTRIM(RTRIM(@Data));
END
GO

Commands completed successfully.

Total execution time: 00:00:00.002

In [19]:
-- # 4.
UPDATE Store
SET Zip_Code = dbo.CleanToNum(Zip_Code);

(45 rows affected)

Total execution time: 00:00:00.004

## **Columna Phone**

<span style="font-size: 18px;">
Para la limpieza de los datos de esta columna reutilizamos la función CleanToNum.
</span>

In [20]:
-- # 5.
UPDATE Store
SET Phone = dbo.CleanToNum(Phone);

(45 rows affected)

Total execution time: 00:00:00.005

## **Columna Email**

<span style="font-size: 18px;">

Para la limpieza de los correos electrónicos, asumimos que un correo válido sigue el formato estándar: user@dominio.com. Durante el proceso, identificamos que algunos correos utilizaban 'at' o '#' en lugar del carácter @, y en otros casos, una coma (,) reemplazaba al punto (.). Estos caracteres fueron corregidos, sustituyéndolos por los signos correspondientes.

</span>

In [21]:
-- # 6.
UPDATE Store 
Set Email = LTRIM(RTRIM(REPLACE(REPLACE(REPLACE(Email, ' at ', '@'), '#', '@'), ',', '.')));

(45 rows affected)

Total execution time: 00:00:00.002

<span style="font-size: 18px;">

Ahora veamos el resultado final de la tabla Store.
</span>

In [22]:
select *from Store;

(45 rows affected)

Total execution time: 00:00:00.005

Store_ID,City,Open_Date,Zip_Code,Phone,Email
1,San Antonio,2003-07-24 21:45:30,78015,2136875667,store1@retail.com
2,San Antonio,2005-10-27 07:01:09,78201,2135167236,store2@retail.com
3,San Antonio,2007-02-01 16:59:13,78112,2123501192,store3@walmart.com
4,Houston,2004-07-17 00:28:08,77001,3129778178,store4@walmart.com
5,Houston,2009-04-23 01:08:37,77002,7136191883,store5@store.com
6,Los Angeles,2006-08-10 19:42:16,90001,2127216342,store6@store.com
7,Phoenix,2007-10-16 08:02:37,60601,7138018755,store7@store.com
8,San Diego,2002-03-10 05:40:48,94023,2125445446,store8@retail.com
9,New York,2007-07-24 04:31:20,10001,3122242101,store9@store.com
10,Philadelphia,2007-05-21 13:40:13,19019,2135196422,store10@walmart.com


## **Resultados y conclusiones**

<span style="font-size: 18px;">

La tabla "Store" ha sido depurada de caracteres innecesarios y los datos se encuentran ahora en los formatos adecuados.

En conclusión, los datos están listos para ser utilizados.

</span>